# 1. Basics of working with candles data and indicators

We have two ways how to get candles data:

1. Directly from ByBit (slow performance, suitable for small data sets)
2. From local database (fast performance, need to perform data sync first)

To run this example, make sure that you:

* Start up your PostgreSQL instance:

```bash
docker-compose up -d db
```

* Apply migrations and sync candles data (e.g. through IPython):

```python
from app import db

await db.start()
await db.apply_migrations()
await db.sync_candles_from_bybit()
```

At first, we need to define some constraints of dataset, which we will operate:

In [1]:
import datetime as dt

from app.core import Timeframe


start_at = dt.datetime(2022, 8, 1)
end_at = dt.datetime(2022, 11, 1)
asset = 'BTCUSD'
timeframe = Timeframe.M5

Let's get candles from ByBit. It's simple, just import the client and call `get_candles` method:

In [2]:
from app import bybit

bybit_candles = await bybit.client.get_candles(asset, timeframe, start_at, end_at)

[18:13:59.281] Downloading candles for BTCUSD[5m]...


26496it [00:14, 1852.66it/s]

[18:14:14.056] Downloaded 26496 candles for BTCUSD[5m]


Make simple check of our data: 

In [3]:
print('Total:', len(bybit_candles))
print('Min:', bybit_candles[0])
print('Max:', bybit_candles[-1])

Total: 26496
Min: {'open_time': datetime.datetime(2022, 8, 1, 0, 0), 'open': 23742.0, 'close': 23897.0, 'low': 23700.0, 'high': 23900.0, 'volume': 8184106.0}
Max: {'open_time': datetime.datetime(2022, 10, 31, 23, 55), 'open': 20454.0, 'close': 20419.5, 'low': 20414.5, 'high': 20456.0, 'volume': 683178.0}


To get candles from DB, we have the same interface as in case of Bybit. But first of that, we need to initialize DB connection through `db.start`:

In [4]:
from app import db

await db.start()

db_candles = await db.get_candles(asset, timeframe, start_at, end_at)

Make same checks that we made for Bybit dataset:

In [5]:
print('Total:', len(db_candles))
print('Min:', db_candles[0])
print('Max:', db_candles[-1])

Total: 26496
Min: {'open_time': datetime.datetime(2022, 8, 1, 0, 0), 'open': 23742.0, 'close': 23897.0, 'low': 23700.0, 'high': 23900.0, 'volume': 8184106.0}
Max: {'open_time': datetime.datetime(2022, 10, 31, 23, 55), 'open': 20454.0, 'close': 20419.5, 'low': 20414.5, 'high': 20456.0, 'volume': 683178.0}


Now it's time to pass data to indicators.

The main goal of indicators design - make them as simple as possible with gaining max performance of native Python (and Numpy in some calculations). They are providing only loop-based interface, no vector calculations. As a result, indicators are less error-prone because we can use the same code for every trading needs (backtesting & live trading). The main drawback - restricted performance (no HFT today, folks)

In [6]:
from app.indicators import *
from app.config import logger

# Use tqdm to check calc performance
from tqdm import tqdm


# Three different type of indicators with the 200-ticks moving window (ticks based on timefrme of dataset, in our case - 5 minutes)
sma = SMA(200)
ema = EMA(200)
wma = WMA(200)

logger.info('Calculating SMA...')
for candle in tqdm(db_candles):
    # We can build dataset of indicator output if we need, indicator does not made it by themself
    val = sma.calculate(candle['close'])
    

logger.info('Calculating EMA...')
for candle in tqdm(db_candles):
    val = ema.calculate(candle['close'])
    

logger.info('Calculating WMA...')
for candle in tqdm(db_candles):
    val = wma.calculate(candle['close'])

[18:14:26.693] Calculating SMA...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26496/26496 [00:00<00:00, 439791.68it/s]

[18:14:26.759] Calculating EMA...



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26496/26496 [00:00<00:00, 3287548.18it/s]

[18:14:26.770] Calculating WMA...



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26496/26496 [00:00<00:00, 53645.39it/s]
